In [1]:
import area_mandelbrot as am
import sampling_alg as sa

import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [27]:
def area(func, bounds, s, i):
    # Initialize the grid and create samples
    rng = np.random.default_rng()
    rng2 = np.random.default_rng()
    re_min, re_max, im_min, im_max = bounds
    area_total = (np.abs(re_min) + np.abs(re_max)) * (np.abs(im_min) + np.abs(im_max))
    samples = func(re_min,re_max,im_min,im_max,rng,s,rng2)

    # Check if the samples are part of the MB-set
    res = am.mandelbrot(samples, i)
    ct_res = Counter(res)

    # Calculate the area
    area_mandel = area_total * (ct_res[i-1] / (sum(ct_res.values())))
    return area_mandel

In [28]:
s = 10 ** 4 # sample size
i = 4 * 10 ** 4 # iterations
bounds = -2, 0.5, -1.2, 1.2 # real-min,max,im-min,max

# Compute area
print(f"Area of the set with PR: {area(sa.sample_pr, bounds, s, i)}")
print(f"Area of the set with PR: {area(sa.sample_lh, bounds, s, i)}")
print(f"Area of the set with PR: {area(sa.sample_ot, bounds, s, i)}")

Area of the set with PR: 1.476
Area of the set with PR: 1.4712
Area of the set with PR: 1.5084


In [5]:
# converge = np.sum(res[res == i])
# print(converge)
# all_my = np.sum(res)
# ratio = (ct_res[i-1] / (sum(ct_res.values()) - ct_res[i-1]))
# ratio